**PART 1 : CONTENT BASED FILTERING**




In [0]:
#IMPORTING LIBRARIES

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer



In [3]:
#UPLOADING DATA FILES(.CSV)
from google.colab import files
uploaded = files.upload()

Saving posts.csv to posts.csv
Saving users.csv to users.csv
Saving views.csv to views.csv


In [0]:
#READING DATA IN PANDAS DATAFRAME

posts = pd.read_csv('posts.csv')
users = pd.read_csv('users.csv')
views = pd.read_csv('views.csv')

MAKING DATA SUITABLE FOR USE

In [8]:
posts.rename(columns ={'_id':'post_id'},inplace=True)
posts


,post_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog
...,...,...,...,...
488,5ed3791976027d35905ccab6,CI/CD using GitHub Actions,Computer Technology|Information Technology,blog
489,5ed3820f76027d35905ccac8,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog
490,5ed3ef4cbd514d602c1530f2,Recommendation Engine,NaN,project
491,5ed415c6bd514d602c15312d,Face Recognition using Transfer Learning.,Science; Technology,skill


In [7]:
users.rename(columns ={'_id':'user_id'},inplace=True)
users

,user_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate
...,...,...,...,...
113,5ed35aa376027d35905cca67,Steve,male,graduate
114,5ed3737976027d35905ccaa2,Himanshu Singh,male,undergraduate
115,5ed3748576027d35905ccaab,Parth Vijay,male,undergraduate
116,5ed3cbcbbd514d602c1530a7,Abhishek Mishra,male,undergraduate


In [9]:
views


,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z
...,...,...,...
1444,5d60098a653a331687083238,5e7a60edcfc8b713f5ac7d82,2020-03-26T19:03:55.284Z
1445,5d60098a653a331687083238,5e7a629ccfc8b713f5ac7d84,2020-03-26T19:01:42.384Z
1446,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,2020-03-26T19:00:25.693Z
1447,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,2020-03-26T18:49:14.721Z


CREATING A NEW DATAFRAME WITH ONLY THE IMPORTANT COLUMNS

In [0]:
df1 = posts[['category',' post_type']]

In [11]:
df1

,category,post_type
0,Plant Biotechnology,blog
1,Artificial Intelligence|Machine Learning|Infor...,blog
2,Operating Systems,blog
3,Drawings,artwork
4,Competition Laws,blog
...,...,...
488,Computer Technology|Information Technology,blog
489,Computer Technology|Computer Application|Infor...,blog
490,NaN,project
491,Science; Technology,skill


In [12]:
df1 = df1[df1['category'].notnull()]
df1

,category,post_type
0,Plant Biotechnology,blog
1,Artificial Intelligence|Machine Learning|Infor...,blog
2,Operating Systems,blog
3,Drawings,artwork
4,Competition Laws,blog
...,...,...
487,Technology,skill
488,Computer Technology|Information Technology,blog
489,Computer Technology|Computer Application|Infor...,blog
491,Science; Technology,skill


In [0]:
df1['category'] = df1['category'].map(lambda x : x.split('|'))

In [0]:
df1['title'] = posts['title']

In [15]:
df1


,category,post_type,title
0,[Plant Biotechnology],blog,hello there
1,"[Artificial Intelligence, Machine Learning, In...",blog,Ml and AI
2,[Operating Systems],blog,What is an Operating System ?
3,[Drawings],artwork,Lord Shiva
4,[Competition Laws],blog,How Competition law evolved?
...,...,...,...
487,[Technology],skill,AWS CLI Setup in Mac
488,"[Computer Technology, Information Technology]",blog,CI/CD using GitHub Actions
489,"[Computer Technology, Computer Application, In...",blog,Configure Docker with Django; PostgreSQL; Pg-a...
491,[Science; Technology],skill,Face Recognition using Transfer Learning.


In [16]:
df1.set_index('title', inplace = True)
df1.head(10)

,category,post_type
title,,
hello there,[Plant Biotechnology],blog
Ml and AI,"[Artificial Intelligence, Machine Learning, In...",blog
What is an Operating System ?,[Operating Systems],blog
Lord Shiva,[Drawings],artwork
How Competition law evolved?,[Competition Laws],blog
Raghavan Committee - 2002.,[Competition Laws],blog
Let's discuss some Case laws!,[Competition Laws],blog
Forms of Cartel.,[Competition Laws],blog
Biodiversity,[Eco System],blog


CONVERTING THE CATEGORY AND POST TYPE INTO A BAG OF WORDS(makes it easy for the inbuilt python function to be applied)

In [0]:
df1['bag_of_words'] = ''
columns = df1.columns
for index, row in df1.iterrows():
    words = ''
    for col in columns:
      if col != ' post_type':
        words = words + ' '.join(row[col])+ ' '
      else:
        words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df1.drop(columns = [col for col in df1.columns if col!= 'bag_of_words'], inplace = True)

In [18]:
df1.head()

,bag_of_words
title,
hello there,Plant Biotechnology blog
Ml and AI,Artificial Intelligence Machine Learning Infor...
What is an Operating System ?,Operating Systems blog
Lord Shiva,Drawings artwork
How Competition law evolved?,Competition Laws blog


In [19]:
count = CountVectorizer()
count_matrix = count.fit_transform(df1['bag_of_words'])

indices = pd.Series(df1.index)
indices[:5]

0                      hello there
1                        Ml and AI
2    What is an Operating System ?
3                       Lord Shiva
4     How Competition law evolved?
Name: title, dtype: object

In [20]:
#USING COSINE SIMILARITY TO FIND SIMILARITY BETWEEN POSTS

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.21821789, 0.33333333, ..., 0.17407766, 0.        ,
        0.17407766],
       [0.21821789, 1.        , 0.21821789, ..., 0.45584231, 0.21821789,
        0.45584231],
       [0.33333333, 0.21821789, 1.        , ..., 0.17407766, 0.        ,
        0.17407766],
       ...,
       [0.17407766, 0.45584231, 0.17407766, ..., 1.        , 0.34815531,
        1.        ],
       [0.        , 0.21821789, 0.        , ..., 0.34815531, 1.        ,
        0.34815531],
       [0.17407766, 0.45584231, 0.17407766, ..., 1.        , 0.34815531,
        1.        ]])

In [0]:
#FUNCTION TO GIVE THE RECOMMENDED POSTS AS OUTPUT FOR A GIVEN POST INPUT

def recommendations(title, cosine_sim = cosine_sim):
    
    recommended = []
    idx = indices[indices == title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 5 most similar posts
    top_5_indexes = list(score_series.iloc[1:6].index)
    
    # populating the list with the titles of the best 5 matching posts
    for i in top_5_indexes:
        recommended.append(list(df1.index)[i])
    print('TITLES:')
    for i in range(0,len(recommended)) :
      
      print((i+1),recommended[i])
    #return recommended

**FINAL CONTENT-BASED RECOMMENDATIONS as per the POST's TITLE**
(Recommend similar posts for the given post)

---




In [22]:
recommendations('Ml and AI')

TITLES:
1 GAN's Part(2)
2 Recommend Systems Machine Learning
3 GAN's INTRODUCTION
4 Machine Learning”&“Operations" (MlOps)
5 Learning...


END OF PART 1

**PART 2: COLLABORATIVE FILTERING**

In [0]:
posts

In [0]:
users

In [0]:
views

In [0]:
views.groupby('user_id')
views

In [0]:
views['val'] = 0
for i in range(len(views['user_id'])):
  views['val'] = 1

In [0]:
mat1 = views.pivot_table(index= 'user_id', columns = 'post_id',values='val')

In [0]:
mat1

In [0]:
mat = views.pivot_table(index= 'user_id', columns = 'post_id',values='val')

In [0]:
mat.fillna(0,inplace=True)

In [30]:
#DATAFRAME WHICH STORES THE INFORMATION ABOUT WHICH USER HAS VIEWED WHICH POSTS
# 1.0 INDICATES THE USER HAS VIEWED THE POST AND 0.0 INDICATES THE USER HAS NOT VIEWED THE POST
mat

post_id,5d62abaa65218653a132c956,5d6d39567fa40e1417a4931c,5d7d23315720533e15c3b1ee,5d7d405e5720533e15c3b1f3,5d80dfbc6c53455f896e600e,5d80e7c16c53455f896e6014,5d80ecfd6c53455f896e601a,5d81323a6c53455f896e6044,5d9b3514979d5962253c2f90,5d9b950768671220a1b2b153,5da745b6019399436815c4cd,5dada695610ba040fbfdf585,5dada734610ba040fbfdf588,5dada798610ba040fbfdf58b,5dada82f610ba040fbfdf58e,5dada89b610ba040fbfdf591,5dadf6ae610ba040fbfdf5c8,5db2a7d099cbb90e4339c734,5db2b91a99cbb90e4339c749,5dbc622a99cbb90e4339c7f6,5dbc631f99cbb90e4339c7fd,5dc065ca24b883670268772f,5dd1751db802e41ed198b680,5dde6a91369b28584ecca156,5ddeb6e80eb5e25a8a07f065,5de179d80eb5e25a8a07f079,5de7971b8eab6401affbb137,5de8d73249e8203ff9219a74,5dea816a42a8854bf6eaba89,5dee9b5042a8854bf6eabaaf,5df64ad1cc709107827fb3dd,5e04fb252a37d20505da2a2b,5e1029f22a37d20505da2a79,5e27002d2a37d20505da2c13,5e281cf0025ae20886de2050,5e2867f6025ae20886de205a,5e294fd1c85ab714a7da6643,5e2d36ebc85ab714a7da66d1,5e2d3fbfc85ab714a7da66d5,5e2d447dc85ab714a7da66d7,...,5ecf23b87023451e66223944,5ecf818376027d35905cbf03,5ecf96e876027d35905cbf46,5ecfa0ca76027d35905cbf57,5ecfafcc76027d35905cbfb0,5ecfb35c76027d35905cbfbc,5ecfffb976027d35905cc0e9,5ed0007a76027d35905cc0ea,5ed09ad076027d35905cc15f,5ed0e20776027d35905cc2fe,5ed0e31a76027d35905cc302,5ed0e65976027d35905cc356,5ed0f0c576027d35905cc3c7,5ed13d2876027d35905cc4c2,5ed141aa76027d35905cc4c9,5ed175f176027d35905cc554,5ed175f476027d35905cc562,5ed1ff0276027d35905cc60d,5ed229d976027d35905cc666,5ed2378276027d35905cc6b5,5ed237d676027d35905cc6bd,5ed2383776027d35905cc6ca,5ed238d376027d35905cc6e5,5ed23a6376027d35905cc73b,5ed23b0776027d35905cc746,5ed23b6276027d35905cc74f,5ed23bb876027d35905cc759,5ed23c1876027d35905cc761,5ed23c8676027d35905cc780,5ed23cb876027d35905cc788,5ed23cf876027d35905cc790,5ed23d4276027d35905cc798,5ed23e4d76027d35905cc7b8,5ed2502b76027d35905cc7db,5ed3476576027d35905cca1d,5ed3791976027d35905ccab6,5ed3820f76027d35905ccac8,5ed3ef4cbd514d602c1530f2,5ed415c6bd514d602c15312d,5ed4cbadbd514d602c1531a6
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5d610ae1653a331687083239,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d618359fc5fcf3bdd9a0910,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d6d2bb87fa40e1417a49315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7c994d5720533e15c3b1e9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [31]:
#USING COSINE_SIMILARITY FUNCTION TO FIND THE SIMILARITY BETWEEN USERS
cosine_sim2 = cosine_similarity(mat)
np.fill_diagonal(cosine_sim2,0)
similar_users = pd.DataFrame(cosine_sim2,index = mat.index)
similar_users.columns = mat.index
similar_users.head()

user_id,5d60098a653a331687083238,5d610ae1653a331687083239,5d618359fc5fcf3bdd9a0910,5d6d2bb87fa40e1417a49315,5d7c994d5720533e15c3b1e9,5d90fc1a641ef364939fb6cc,5d9a2982979d5962253c2f81,5de50d768eab6401affbb135,5ded443342a8854bf6eaba9a,5deeef6142a8854bf6eabab9,5defd51362624b0135ea9fd2,5df11ee1ee4bb5252b4f5344,5df20f1fee4bb5252b4f5351,5df24eeaee4bb5252b4f5366,5df3f8f2ee4bb5252b4f5393,5df49b32cc709107827fb3c7,5e0d5ce12a37d20505da2a5d,5e1ef04c2a37d20505da2b8b,5e26bb482a37d20505da2c10,5e3563348d344822fed4d13a,5e35a5ed8d344822fed4d13e,5e365e758d344822fed4d144,5e37b16ceab55d319938a6f8,5e4195c3eab55d319938a82c,5e4ce251f5561b1994c8e40d,5e5855ced701ab08af792b51,5e5af599d701ab08af792b63,5e5b4511d701ab08af792b6d,5e5dfbbefbc8805f69e02c91,5e783375cfc8b713f5ac7c5b,5e78ce84cfc8b713f5ac7cee,5e7b30a6cfc8b713f5ac7d95,5e7cf05bcfc8b713f5ac7db7,5e7dde87a3258347b42f2108,5e7de336a3258347b42f210e,5e7def80a3258347b42f2124,5e7f4ad1a3258347b42f2155,5e822b86a3258347b42f2360,5e8309c4a3258347b42f23f4,5e840a75a3258347b42f2437,...,5eb4f73910426255a7aaa0cc,5eb508ae10426255a7aaa12c,5eb7aeccfd92f539c465ddd7,5ebc2ec4514aab59896bcc84,5ebcf2f2514aab59896bcd00,5ec11abc74f7660d73aa0fbd,5ec2204374f7660d73aa100f,5ec3ba5374f7660d73aa1201,5ec553e2f2781131cc7e5138,5ecb979eeaff6b0c3a58a4f0,5ecd5a5c7023451e662235c1,5ecdf7397023451e66223784,5ece02427023451e66223861,5ecf6e6576027d35905cbeec,5ecfa94676027d35905cbf78,5ecfaa0076027d35905cbf83,5ecfbcb376027d35905cbfe5,5ecfdaa976027d35905cc02c,5ed09e8576027d35905cc164,5ed0bfc176027d35905cc1f8,5ed0cfc676027d35905cc26a,5ed0d9bc76027d35905cc2c2,5ed0defa76027d35905cc2de,5ed0dfec76027d35905cc2ec,5ed0fc0976027d35905cc448,5ed12ae476027d35905cc4a1,5ed13bb076027d35905cc4ba,5ed142cd76027d35905cc4cd,5ed232b976027d35905cc689,5ed237ee76027d35905cc6c5,5ed2399b76027d35905cc70f,5ed24d5a76027d35905cc7d7,5ed2672576027d35905cc7f6,5ed2edf176027d35905cc961,5ed350ed76027d35905cca2c,5ed35aa376027d35905cca67,5ed3737976027d35905ccaa2,5ed3748576027d35905ccaab,5ed3cbcbbd514d602c1530a7,5ed3e898bd514d602c1530de
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,0.000000,0.405826,0.140901,0.268175,0.349760,0.0,0.047782,0.186030,0.0,0.272430,0.229154,0.000000,0.084708,0.117041,0.238909,0.213318,0.047782,0.231681,0.0,0.238541,0.132712,0.0,0.081088,0.110347,0.067574,0.067574,0.331502,0.151099,0.135147,0.047782,0.0,0.117041,0.219797,0.000000,0.033787,0.047782,0.112623,0.095564,0.067574,0.0,...,0.082761,0.095564,0.047782,0.067574,0.095564,0.047782,0.137934,0.259033,0.067574,0.122245,0.0,0.117041,0.095564,0.120880,0.000000,0.000000,0.067574,0.039014,0.067574,0.117041,0.095564,0.067574,0.060440,0.047782,0.067574,0.067574,0.047782,0.000000,0.101361,0.082761,0.082761,0.047782,0.000000,0.095564,0.110347,0.033787,0.067574,0.067574,0.067574,0.067574
5d610ae1653a331687083239,0.405826,0.000000,0.163340,0.180937,0.272184,0.0,0.061546,0.219649,0.0,0.319005,0.230997,0.087039,0.128947,0.050252,0.215410,0.193952,0.061546,0.348155,0.0,0.221906,0.208928,0.0,0.121854,0.071067,0.000000,0.087039,0.258443,0.116775,0.174078,0.061546,0.0,0.050252,0.196946,0.000000,0.043519,0.000000,0.087039,0.000000,0.087039,0.0,...,0.071067,0.061546,0.061546,0.000000,0.061546,0.000000,0.035533,0.261116,0.087039,0.118094,0.0,0.050252,0.000000,0.038925,0.087039,0.000000,0.087039,0.000000,0.000000,0.050252,0.000000,0.000000,0.038925,0.000000,0.000000,0.000000,0.061546,0.000000,0.043519,0.071067,0.071067,0.000000,0.000000,0.000000,0.071067,0.043519,0.000000,0.043519,0.000000,0.000000
5d618359fc5fcf3bdd9a0910,0.140901,0.163340,0.000000,0.118217,0.202362,0.0,0.000000,0.095673,0.0,0.152845,0.184463,0.000000,0.118812,0.000000,0.036860,0.077440,0.000000,0.000000,0.0,0.163572,0.318511,0.0,0.125109,0.000000,0.000000,0.000000,0.215353,0.000000,0.000000,0.147442,0.0,0.000000,0.117954,0.208514,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085126,0.139010,0.000000,0.377217,0.0,0.120386,0.294884,

In [0]:
df2 = views.copy()

In [0]:
df2

In [0]:
# FUNCTION TO FIND SIMILAR USERS

def find_n_neighbours(df2,n):
    order = np.argsort(df2.values, axis=1)[:, :n]
    df2 = df2.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df2

In [36]:
# TOP 30 SIMILAR USER TO A GIVEN USER 

sim_user_30_u = find_n_neighbours(similar_users,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,5d610ae1653a331687083239,5d7c994d5720533e15c3b1e9,5e5af599d701ab08af792b63,5deeef6142a8854bf6eabab9,5d6d2bb87fa40e1417a49315,5ec3ba5374f7660d73aa1201,5df3f8f2ee4bb5252b4f5393,5e3563348d344822fed4d13a,5e1ef04c2a37d20505da2b8b,5defd51362624b0135ea9fd2,5e7cf05bcfc8b713f5ac7db7,5df49b32cc709107827fb3c7,5de50d768eab6401affbb135,5ea3f6dc10426255a7aa9aff,5e5b4511d701ab08af792b6d,5ea890e410426255a7aa9c16,5e94248aa3258347b42f2804,5e941316a3258347b42f27f4,5d618359fc5fcf3bdd9a0910,5ec2204374f7660d73aa100f,5e5dfbbefbc8805f69e02c91,5e35a5ed8d344822fed4d13e,5ecb979eeaff6b0c3a58a4f0,5ecf6e6576027d35905cbeec,5e99b0d4a3258347b42f2bf0,5ecdf7397023451e66223784,5eaa137910426255a7aa9c41,5e7b30a6cfc8b713f5ac7d95,5ed0bfc176027d35905cc1f8,5df24eeaee4bb5252b4f5366
5d610ae1653a331687083239,5d60098a653a331687083238,5e1ef04c2a37d20505da2b8b,5deeef6142a8854bf6eabab9,5d7c994d5720533e15c3b1e9,5ec3ba5374f7660d73aa1201,5e5af599d701ab08af792b63,5defd51362624b0135ea9fd2,5e3563348d344822fed4d13a,5de50d768eab6401affbb135,5df3f8f2ee4bb5252b4f5393,5e35a5ed8d344822fed4d13e,5e7cf05bcfc8b713f5ac7db7,5df49b32cc709107827fb3c7,5d6d2bb87fa40e1417a49315,5e5dfbbefbc8805f69e02c91,5e9a8013a3258347b42f2c29,5d618359fc5fcf3bdd9a0910,5e93471ea3258347b42f279f,5df20f1fee4bb5252b4f5351,5e37b16ceab55d319938a6f8,5ecb979eeaff6b0c3a58a4f0,5e880f02a3258347b42f252c,5e5b4511d701ab08af792b6d,5eb2cad810426255a7aaa072,5e9c146ba3258347b42f2cbd,5e845fbaa3258347b42f2450,5e5855ced701ab08af792b51,5e8309c4a3258347b42f23f4,5ec553e2f2781131cc7e5138,5e902793a3258347b42f2733
5d618359fc5fcf3bdd9a0910,5ecb979eeaff6b0c3a58a4f0,5e35a5ed8d344822fed4d13e,5ed232b976027d35905cc689,5ece02427023451e66223861,5eb2cad810426255a7aaa072,5ed0bfc176027d35905cc1f8,5e5af599d701ab08af792b63,5ed3cbcbbd514d602c1530a7,5ecfaa0076027d35905cbf83,5e7dde87a3258347b42f2108,5ed09e8576027d35905cc164,5ed0d9bc76027d35905cc2c2,5ed0fc0976027d35905cc448,5ed3e898bd514d602c1530de,5ed142cd76027d35905cc4cd,5ed3737976027d35905ccaa2,5ed12ae476027d35905cc4a1,5d7c994d5720533e15c3b1e9,5defd51362624b0135ea9fd2,5ed350ed76027d35905cca2c,5e3563348d344822fed4d13a,5d610ae1653a331687083239,5deeef6142a8854bf6eabab9,5ed0dfec76027d35905cc2ec,5ed24d5a76027d35905cc7d7,5ed0cfc676027d35905cc26a,5ed2edf176027d35905cc961,5e783375cfc8b713f5ac7c5b,5e88156ca3258347b42f2542,5d60098a653a331687083238
5d6d2bb87fa40e1417a49315,5e3563348d344822fed4d13a,5defd51362624b0135ea9fd2,5df49b32cc709107827fb3c7,5ec3ba5374f7660d73aa1201,5d7c994d5720533e15c3b1e9,5e5af599d701ab08af792b63,5de50d768eab6401affbb135,5e7cf05bcfc8b713f5ac7db7,5deeef6142a8854bf6eabab9,5ec2204374f7660d73aa100f,5e35a5ed8d344822fed4d13e,5e1ef04c2a37d20505da2b8b,5d60098a653a331687083238,5eb508ae10426255a7aaa12c,5ec11abc74f7660d73aa0fbd,5eb2cad810426255a7aaa072,5ecfbcb376027d35905cbfe5,5d610ae1653a331687083239,5df20f1fee4bb5252b4f5351,5d618359fc5fcf3bdd9a0910,5ecb979eeaff6b0c3a58a4f0,5e37b16ceab55d319938a6f8,5ecdf7397023451e66223784,5e7b30a6cfc8b713f5ac7d95,5ed0bfc176027d35905cc1f8,5ecfdaa976027d35905cc02c,5ed232b976027d35905cc689,5e4195c3eab55d319938a82c,5ed350ed76027d35905cca2c,5e942f7ba3258347b42f280e
5d7c994d5720533e15c3b1e9,5e3563348d344822fed4d13a,5ec3ba5374f7660d73aa1201,5d6d2bb87fa40e1417a49315,5df49b32cc709107827fb3c7,5e5af599d701ab08af792b63,5defd51362624b0135ea9fd2,5d60098a653a331687083238,5e35a5ed8d344822fed4d13e,5deeef6142a8854bf6eabab9,5e37b16ceab55d319938a6f8,5de50d768eab6401affbb135,5df20f1fee4bb5252b4f5351,5e7cf05bcfc8b713f5ac7db7,5d610ae1653a331687083239,5eb2cad810426255a7aaa072,5ec2204374f7660d73aa100f,5e1ef04c2a37d20505da2b8b,5d618359fc5fcf3bdd9a0910,5ecb979eeaff6b0c3a58a4f0,5eb508ae10426255a7aaa12c,5ed0cfc676027d35905cc26a,5ed2edf176027d35905cc961,5ec11abc74f7660d73aa0fbd,5ed350ed76027d35905cca2c,5e4195c3eab55d319938a82c,5ecdf7397023451e66223784,5ecfdaa976027d3

In [56]:
views_users = pd.merge(views, users, on='user_id')
views_users

,user_id,post_id,timestamp,val,name,gender,academics
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,1,Niriksha Sharma,female,undergraduate
1,5df49b32cc709107827fb3c7,5ecd5d417023451e662235c5,2020-05-26T20:11:07.153Z,1,Niriksha Sharma,female,undergraduate
2,5df49b32cc709107827fb3c7,5dee9b5042a8854bf6eabaaf,2020-05-25T09:58:52.351Z,1,Niriksha Sharma,female,undergraduate
3,5df49b32cc709107827fb3c7,5ecb72c0eaff6b0c3a58a48e,2020-05-25T07:27:20.699Z,1,Niriksha Sharma,female,undergraduate
4,5df49b32cc709107827fb3c7,5ec3c09274f7660d73aa1229,2020-05-25T07:26:57.220Z,1,Niriksha Sharma,female,undergraduate
...,...,...,...,...,...,...,...
1444,5e7de336a3258347b42f210e,5e5e3b35fbc8805f69e02c9e,2020-03-27T11:35:13.977Z,1,Suman Ray,female,graduate
1445,5e7de336a3258347b42f210e,5e5facf649d7f322c033e02b,2020-03-27T11:35:06.956Z,1,Suman Ray,female,graduate
1446,5e7de336a3258347b42f210e,5e7de250a3258347b42f210a,2020-03-27T11:35:02.354Z,1,Suman Ray,female,graduate
1447,5e7dde87a3258347b42f2108,5e79f3fccfc8b713f5ac7d54,2020-03-27T11:09:38.898Z,1,WAEL Onsy,male,graduate


In [57]:
views_users_posts = pd.merge(views_users, posts, on= 'post_id')
views_users_posts

,user_id,post_id,timestamp,val,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,1,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,2020-05-22T20:11:32.317Z,1,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,2020-05-22T20:01:37.309Z,1,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,2020-05-22T20:10:41.100Z,1,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,2020-05-22T20:08:45.061Z,1,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
...,...,...,...,...,...,...,...,...,...,...
1442,5e899be7a3258347b42f25d0,5e3ab644eab55d319938a72d,2020-04-05T08:57:02.583Z,1,Srinidhi A,female,undergraduate,Travelling,Photography,artwork
1443,5e78ce84cfc8b713f5ac7cee,5e78d6dccfc8b713f5ac7cf4,2020-04-05T05:44:03.998Z,1,Fauziya Shaikh,female,undergraduate,Childhood Sketch,Drawings|Watercolours,artwork
1444,5e880f02a3258347b42f252c,5e783590cfc8b713f5ac7c5d,2020-04-04T04:44:54.488Z,1,Keshant Sharma,male,graduate,Quarantine days,Painting,artwork
1445,5e783375cfc8b713f5ac7c5b,5e787636cfc8b713f5ac7cbe,2020-03-28T11:45:05.321Z,1,Bhargav Prakash,male,undergraduate,Sunsets in Ottawa,Painting,artwork


In [0]:
views_users_posts.drop(columns=['timestamp', 'gender', 'academics', 'val', 'category', ' post_type'], inplace=True)

In [80]:
# DROPPING EXTRA COLUMNS AND MAKING THE DATAFRAME AESTHETIC

views_users_posts

,user_id,post_id,name,title
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,Niriksha Sharma,Save Earth.
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,Varun Chowhan,Save Earth.
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,Ilupeju Ayokunnumi,Save Earth.
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,thesocialcomment,Save Earth.
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,siddharth saxena,Save Earth.
...,...,...,...,...
1442,5e899be7a3258347b42f25d0,5e3ab644eab55d319938a72d,Srinidhi A,Travelling
1443,5e78ce84cfc8b713f5ac7cee,5e78d6dccfc8b713f5ac7cf4,Fauziya Shaikh,Childhood Sketch
1444,5e880f02a3258347b42f252c,5e783590cfc8b713f5ac7c5d,Keshant Sharma,Quarantine days
1445,5e783375cfc8b713f5ac7c5b,5e787636cfc8b713f5ac7cbe,Bhargav Prakash,Sunsets in Ottawa


In [0]:
# FUNCTION TO GET SIMILAR POSTS BETWEEN 2 USERS

def get_user_similar_posts( user1, user2 ):
    common_posts = views_users_posts[views_users_posts.user_id == user1].merge(
    views_users_posts[views_users_posts.user_id == user2],
    on = "post_id",
    how = "inner" )
    return common_posts.merge( posts, on = 'post_id' )

In [68]:
# THE TWO INPUTS ARE THE USER_IDs AND THESE 2 USERS HAVE THE FOLLOWING POSTS IN COMMON

a = get_user_similar_posts('5df49b32cc709107827fb3c7'	,'5d7c994d5720533e15c3b1e9')
a = a.loc[ : , ['title']]
a.head()

,title
0,Save Earth.
1,"Machine Learning”&“Operations"" (MlOps)"
2,EID MUBARAK
3,Dakrai Artwork.
4,My First Animated Post.


In [0]:
views_users_posts = views_users_posts.astype({'post_id':str})
post_user = views_users_posts.groupby(by = 'user_id')['post_id'].apply(lambda x:','.join(x))

In [0]:
post_user

In [0]:
#FUNCTION TO RECOMMEND POSTS BASED ON OTHER USERS VIEWING
def recommend_user(user):
    post_seen_by_user = mat1.columns[mat1[mat1.index==user].notna().any()].tolist()
    a = sim_user_30_u[sim_user_30_u.index==user].values
    b = a.squeeze().tolist()
    d = post_user[post_user.index.isin(b)]
    l = ','.join(d.values)
    posts_seen_by_similar_users = l.split(',')
    posts_under_consideration = list(set(posts_seen_by_similar_users)-set(list(map(str, post_seen_by_user))))
    posts_under_consideration = list(map(str, posts_under_consideration))
    score = []
    for item in posts_under_consideration:
        c = mat.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        #avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similar_users.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'post_id':posts_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    post_Name = top_5_recommendation.merge(posts, how='inner', on='post_id')
    post_Names = post_Name.title.values.tolist()
    return post_Names

**FINAL CONTENT-BASED RECOMMENDATIONS as per the POST's TITLE** (Recommend posts for the given user)

In [101]:
#ENTERING USER_ID TO GET ALL THE RECOMMENDED POSTS
recommend_user('5d60098a653a331687083238')

['Art Expo 2020',
 'Faith in yourself',
 'Save Earth.',
 'Designing Cmos circuit from Boolean expressions (Microwind)',
 'Sky never disappoints me!']